In [ ]:
import pandas as pd 
import numpy as np
from docx.api import Document
import re
import json

# templates = [["header",1,"Metro is making more service changes.","Metro está haciendo más cambios en sus servicios.","Metro正在進行更多服務調整。","Metro hiện đang thực hiện nhiều thay đổi về dịch vụ.","메트로 서비스가 더욱 새롭게단장하고 있습니다.","メトロのサービスが変更されます。","Metro-ն կրկին փոփոխություններ է իրականացնում ծառայությունների մեջ:","Metro вносит дополнительные изменения в схемы движения."]]
# templates = ["header",1],["summary",1],["details",1],["end",1]
# final_template = pd.DataFrame(templates,columns=["section","order","en","es","zh-TW","vi","ko","ja","hy","ru"])


In [ ]:
document = Document('data/202109shakeup.docx')
table = document.tables[0]
data = [[cell.text.replace("\n"," ").replace('"','').replace('" ','').lstrip() for cell in row.cells] for row in table.rows]

df = pd.DataFrame(data)
new_header = df.iloc[0]
df = df[1:] 
df.columns = new_header
# print(df.columns)
df = df.rename(columns={'English':'en','Spanish':'es','Chinese (Traditional)':'zh-TW','Korean':'ko','Vietnamese':'vi','Japanese':'ja','Russian':'ru','Armenian':'hy'})
# df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)

df = df.replace(' +',r' ',regex=True)
df = df.replace('"',r'',regex=True)
# df.to_json('test.json')
# df.to_csv('test.csv')
df.head()

final_df = pd.DataFrame(columns=["section","order","line","altline","en","es","zh-TW","vi","ko","ja","hy","ru","new-schedule","current-schedule"])

## Populating the data

### 2.1 Adding the `Summary` sections

In [ ]:
def reset_final_df():
    return pd.DataFrame(columns=["section","order","line","altline","en","es","zh-TW","vi","ko","ja","hy","ru","new-schedule","current-schedule"])

In [ ]:
header1 = df.loc[(df['en'].str.contains('\u2013') == False) & (df['en'].str.contains('Metro is making service'))]
header1 = header1.assign(section='header')
header1 = header1.assign(order='1')

header2 = df.loc[(df['en'].str.contains('\u2013') == False) & (df['en'].str.contains('New schedules start'))]
header2 = header2.assign(section='header')
header2 = header2.assign(order='2')

if not final_df.empty:
    final_df = reset_final_df()

final_df = final_df.append(header1)
final_df = final_df.append(header2)

final_df

In [ ]:
final_df

In [ ]:
# th = df[df['en'].str.contains('Starting on'):df['en'].str.contains('We’re ')]
th = df.loc[(df['en'].str.contains('\u2013') == False) & (df.index < 30) & (df['en'].str.contains('We’re modify') == False)]

# th = th.loc[th['section']]
# df.index.rename("oid")



th = th.assign(section='summary')

### RIP: code to split based on `:`

th['order'] = ''
# th['en'] = th['en'].str.split(':')
# th = th.explode('en')
th_count = th.shape[0]
for i in range(0,th_count):
    th['order'].values[i] = i

###

    # th = th['order'].values[i] = i
# for row in th.itertuples():
#     th.at[row.Index,'Index'].assign(order=row.Index)

# th = th.set_index(['index','section']).apply(lambda x: x.str.split(':').explode())
# .reset_index()

# th = th.concat([Series(row['var2'], row['var1'].split(','))              
#                     for _, row in a.iterrows()]).reset_index()


# for i in range(1,th_count):
#     print(i)
# th.loc[:,'section'] = 'summary'
# th.loc[:,'order'] = 1
# th = th.set_index('order',drop=False)


# th = th.reset_index(inplace=True)
# th
# th
# final_df = final_df.append(th).copy()
# final_df
th



final_df = final_df.append(th)
final_df

In [ ]:
detail_header = df.loc[(df['en'].str.contains('\u2013') == False) & (df.index < 20) & (df['en'].str.contains('We’re modify') == True)]

detail_header = detail_header.assign(section='details')
detail_header = detail_header.assign(order=0)

final_df = final_df.append(detail_header)
detail_header
# final_df.to_json('final_takeone.json',orient='records')

In [ ]:
detail_header

### 2.2. Adding the `details`/lines section

First we will get all the lines in:

In [ ]:
# data = []
# lines = []
# keys = None
lines_df = pd.read_csv('data/mybus-sep-2021 - Lines.csv', index_col=0)
lines_takeone_df = df.loc[(df['en'].str.contains('\u2013')) & (df['en'].str.contains('B Line, D Line') == False)]


# lines_takeone_df = lines_takeone_df.index.rename("oid")
# lines_takeone_df = lines_takeone_df.set_index('order')


df.loc[(df['en'].str.contains('\u2013') == False) & (df.index < 30)]

lines_df['AltLine'] = lines_df.AltLine.fillna(0).astype(int)
all_lines = lines_df[['Line Label',"AltLine"]]
# all_lines

lines_count = all_lines.shape[0]

all_lines['order'] = ''
# all_lines['line'] = 

for i in range(0,lines_count):
    all_lines['order'].values[i] = i+1
all_lines.reset_index(inplace=True)
all_lines = all_lines.rename(columns={"Line Label":"line_label","Line Number":"line"})
# all_lines.index.rename('line', inplace=True)
all_lines.head(4)



In [ ]:
line = lines_takeone_df.en.str.split('–')

lines_takeone_df['line'] = lines_takeone_df.en.str.split('–').str[0]

lines_takeone_df = lines_takeone_df.assign(oid=lines_takeone_df.line.str.split('/')).explode('oid')
# lines_takeone_df = lines_takeone_df.drop((lines_takeone_df.duplicated(subset=['line'])))


dupes = lines_takeone_df.loc[(lines_takeone_df.duplicated(subset=['line']))]
lines_takeone_df = lines_takeone_df.drop_duplicates(subset=['line'])
lines_takeone_df = lines_takeone_df[lines_takeone_df["line"].str.contains("/")==False]


lines_takeone_df


# lines_takeone_df['line'] = lines_takeone_df.loc[(lines_takeone_df.duplicated(subset=['line']))]


# dupes = lines_takeone_df.duplicated(subset=['line'])
# dupes


# lines_takeone_df = lines_takeone_df.set_index('line')
# lines_takeone_df.tail()
# result = pd.merge(all_lines, lines_takeone_df, on=["line","line"])


# lines_takeone_df


# result = pd.concat([all_lines,lines_takeone_df],axis=1)
# result = all_lines.join(lines_takeone_df,how="right")
# result = pd.merge(all_lines,lines_takeone_df,how="outer",on=["line","line"])

# lines_takeone_df.head()
# all_lines.head()
# lines_takeone_df.head(2)
# result

In [ ]:
dupes['line'] = dupes['line'].str.split('/')
dupes = dupes.explode('line')


lines_takeone_df = lines_takeone_df.append(dupes)

In [ ]:
all_lines['line'] = all_lines['line'].astype(int)
# lines_takeone_df
lines_takeone_df['line'] = lines_takeone_df['line'].astype(int)

merged_lines = all_lines.merge(lines_takeone_df, on='line',how='outer')

# merged_lines.reset_index()
merged_lines = merged_lines.assign(section='details')
merged_lines



In [ ]:

final_df = final_df.append(merged_lines)

In [ ]:
rail_df = df.loc[(df['en'].str.contains('\u2013')) & (df['en'].str.contains('B Line, D Line') == True)]
end_lines = len(merged_lines) +1
# print(end_lines)
rail_df = rail_df.assign(section='details')
rail_df = rail_df.assign(order=end_lines)

final_df = final_df.append(rail_df)

In [ ]:
end1 = df.loc[(df['en'].str.contains('For more information '))]
end1 = end1.assign(section='end')
end1 = end1.assign(order=1)

end2 = df.loc[(df['en'].str.contains('\\* M'))]
end2 = end2.assign(order=2)
end2 = end2.assign(section='end')
end2


final_df = final_df.append(end1)
final_df = final_df.append(end2)


In [ ]:
final_df.head(10)

In [ ]:
final_df

In [ ]:
# all_lines.set_index('Line Label')
# all_lines = lines_df

# all_lines = lines_df[[lines_df['Line Label'],lines_df['AltLine'].fillna(0).astype(int)]]
# lines_df = lines_df[['Line Label','AltLine']]
# lines_df

final_df

### Final Output

In [ ]:
languages = ['en','es','zh-TW','vi','ko','ja','hy','ru']
DATA_OUTPUT_PATH = "./data/"
for i in languages:
    final_final_df = final_df[['section','order', i,'line', 'new-schedule', 'current-schedule']].copy()
    final_final_df = final_final_df.rename(columns={i: 'content'})
    final_final_df.to_json(DATA_OUTPUT_PATH + 'takeone-' + i + '.json',orient='records')
    print('Takeone created for: ' + i)
